In [14]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import trend_utils as tu
from regridding import regrid_to_loca
from utils import loca_gard_mapping, trend_metric_ids as metric_ids

In [15]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [16]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="30GiB",
    walltime="00:20:00",
)

cluster.scale(jobs=20)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.95:37051,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Fit trend

In [17]:
# Fit all
for metric_id in metric_ids:
    tu.trend_fit_all(metric_id)

## Regridding

In [18]:
%%time
# Regrid to LOCA using NN
method = "nearest"

for metric_id in metric_ids:
    # Save path
    store_path = f"{project_data_path}/trends/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/trends/original_grids/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/trends/original_grids/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        try:
            # Check if done
            file_name = file.split('/')[-1]
            store_name = file_name.replace(".nc", f"_{method}.nc")
            if not os.path.exists(f"{store_path}/{store_name}"):
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid_to_loca(ds_tmp, method=method)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
        except:
            print(file)

/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/trends/original_grids/avg_tas/GARD-LENS_ecearth3_r130i1p1f1_historical_1950-2014.nc
/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024/trends/original_grids/avg_tas/GARD-LENS_ecearth3_r130i1p1f1_ssp370_2015-2100.nc
CPU times: user 545 ms, sys: 156 ms, total: 701 ms
Wall time: 1.63 s
